In [1]:
import jieba
import jieba.analyse
import csv
import pandas as pd
from datetime import datetime,timedelta
jieba.set_dictionary('bda2019_dataset/dict.idkrsi.txt')
from tqdm import tqdm

In [2]:
company='聯發科'
# company2='台塑化'
# companyname='台塑'
# companyrelated='聯發科|系統晶片|無線通|控制晶片|絡達|創發|奕力|晨星|擎發|立錡'

In [3]:
#處理三年股價資料

df2018=pd.read_csv("bda2019_dataset/TWSE2018.csv", encoding="utf-8")
df2017=pd.read_csv("bda2019_dataset/TWSE2017.csv", encoding="utf-8")
df2016=pd.read_csv("bda2019_dataset/TWSE2016.csv", encoding="utf-8")
df_tgg = pd.concat([df2018,df2017,df2016],axis=0, ignore_index=True)  #三個dataframe串起來
# df_tgg = pd.concat([df2017,df2016],axis=0, ignore_index=True)  #兩個dataframe串起來
df_tgg=df_tgg[df_tgg['證券代碼'].str.contains(company)]#找有提到公司的文章
df_tgg=df_tgg.reset_index(drop=True)  #重新Index
df_tgg['年月日']=pd.to_datetime(df_tgg['年月日'])  #把日期轉成datefime格式

df_tgg['收盤價(元)']=df_tgg['收盤價(元)'].astype(str).str.replace(',','').astype('float64')  #把價格轉成float
df_tgg['最高價(元)']=df_tgg['最高價(元)'].astype(str).str.replace(',','').astype('float64')  #把價格轉成float

In [4]:
df_tgg.head(3)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,2454 聯發科,2018-12-28,223.5,229.5,223.5,229.5,"3,415","774,673","1,904","1,591,506",13.4,1.34
1,2454 聯發科,2018-12-27,224.5,227.0,223,225.0,"2,969","668,855","2,189","1,591,506",13.13,1.32
2,2454 聯發科,2018-12-26,222,223.5,220,220.0,897,"198,854",793,"1,591,506",12.84,1.29


In [5]:
#只選用news的文章
#保留特定公司的文章

news=pd.read_csv("bda2019_dataset/news2.csv",encoding="utf-8")
# forum=pd.read_csv("bda2019_dataset/forum2.csv",encoding='utf-8')
# bbs=pd.read_csv("bda2019_dataset/bbs2.csv",encoding='utf-8')
# news=news.drop(columns=['page_url'])
# forum=forum.drop(columns=['comment_count','page_url'])
# bbs=bbs.drop(columns=['comment_count','page_url'])
# news['content'].fillna(value='哈哈', inplace=True)  #把空白補成特定文字
# forum['content'].fillna(value='哈哈', inplace=True)
# bbs['content'].fillna(value='哈哈', inplace=True)
# tnews = pd.concat([news,forum,bbs],axis=0, ignore_index=True)
# tnews= tnews.reset_index(drop=True)
# tnews=tnews[tnews['content'].str.contains(company)]  #搜尋特定文字的文章，有的話保留
# tnews=tnews.reset_index(drop=True)  #重新Index
# tnews['post_time']=pd.to_datetime(tnews['post_time'])  #把post_time變成datetime格式

news['content'].fillna(value='哈哈', inplace=True)  #只用news
tnews=news[news['content'].str.contains(company)]
tnews=tnews.reset_index(drop=True)  #重新Index
tnews['post_time']=pd.to_datetime(tnews['post_time'])

In [6]:
len(tnews)

5179

In [7]:
df_tgg['highpriceshift'] = df_tgg['收盤價(元)'].shift(periods=1)  # 移動三格 可做兩row 上下比較
df_tgg.head(3)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,highpriceshift
0,2454 聯發科,2018-12-28,223.5,229.5,223.5,229.5,"3,415","774,673","1,904","1,591,506",13.4,1.34,NaN
1,2454 聯發科,2018-12-27,224.5,227.0,223,225.0,"2,969","668,855","2,189","1,591,506",13.13,1.32,229.5
2,2454 聯發科,2018-12-26,222,223.5,220,220.0,897,"198,854",793,"1,591,506",12.84,1.29,225.0


In [8]:
#算漲跌率
df_tgg['rate']=(df_tgg['highpriceshift']-df_tgg['收盤價(元)'])/df_tgg['收盤價(元)']
df_tgg.head(5)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,highpriceshift,rate
0,2454 聯發科,2018-12-28,223.5,229.5,223.5,229.5,"3,415","774,673","1,904","1,591,506",13.4,1.34,NaN,NaN
1,2454 聯發科,2018-12-27,224.5,227.0,223,225.0,"2,969","668,855","2,189","1,591,506",13.13,1.32,229.5,0.020000
2,2454 聯發科,2018-12-26,222,223.5,220,220.0,897,"198,854",793,"1,591,506",12.84,1.29,225.0,0.022727
3,2454 聯發科,2018-12-25,221,222.0,219,221.0,"1,528","337,471","1,229","1,591,506",12.9,1.29,220.0,-0.004525
4,2454 聯發科,2018-12-24,223.5,225.5,222.5,225.0,"1,674","375,607","1,173","1,591,506",13.13,1.32,221.0,-0.017778


In [9]:
#把空白rate去掉
df_tgg=df_tgg[3:]
df_tgg=df_tgg.reset_index(drop=True)
df_tgg.head(5)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,highpriceshift,rate
0,2454 聯發科,2018-12-25,221,222.0,219,221.0,"1,528","337,471","1,229","1,591,506",12.9,1.29,220.0,-0.004525
1,2454 聯發科,2018-12-24,223.5,225.5,222.5,225.0,"1,674","375,607","1,173","1,591,506",13.13,1.32,221.0,-0.017778
2,2454 聯發科,2018-12-22,223,223.5,221.5,223.0,923,"205,274",860,"1,591,506",13.02,1.31,225.0,0.008969
3,2454 聯發科,2018-12-21,224,226.0,221.5,223.5,"5,269","1,177,829","2,560","1,591,506",13.05,1.31,223.0,-0.002237
4,2454 聯發科,2018-12-20,229.5,231.0,223.5,225.0,"9,042","2,049,881","5,840","1,591,506",13.13,1.32,223.5,-0.006667


In [10]:
#標註漲跌
# df_tgg['label']='平'
# df_tgg.loc[df_tgg['rate']>=0.04, 'label'] = '漲'
# df_tgg.loc[df_tgg['rate']<=-0.04, 'label'] = '跌'
# df_tgg.loc[(df_tgg['rate']<=0.004) & (df_tgg['rate']>=-0.004), 'label']='不上不下'
# df_tgg.head(10)

df_tgg['label']='平'
df_tgg.loc[df_tgg['rate']>=0.01, 'label'] = '漲'
df_tgg.loc[df_tgg['rate']<=-0.01, 'label'] = '跌'
df_tgg.head(10)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,highpriceshift,rate,label
0,2454 聯發科,2018-12-25,221,222.0,219,221.0,"1,528","337,471","1,229","1,591,506",12.9,1.29,220.0,-0.004525,平
1,2454 聯發科,2018-12-24,223.5,225.5,222.5,225.0,"1,674","375,607","1,173","1,591,506",13.13,1.32,221.0,-0.017778,跌
2,2454 聯發科,2018-12-22,223,223.5,221.5,223.0,923,"205,274",860,"1,591,506",13.02,1.31,225.0,0.008969,平
3,2454 聯發科,2018-12-21,224,226.0,221.5,223.5,"5,269","1,177,829","2,560","1,591,506",13.05,1.31,223.0,-0.002237,平
4,2454 聯發科,2018-12-20,229.5,231.0,223.5,225.0,"9,042","2,049,881","5,840","1,591,506",13.13,1.32,223.5,-0.006667,平
5,2454 聯發科,2018-12-19,232.5,236.0,232.5,236.0,"2,712","636,109","1,752","1,591,506",13.78,1.38,225.0,-0.046610,跌
6,2454 聯發科,2018-12-18,234,236.5,233.5,234.5,"2,413","566,781","1,489","1,591,506",13.69,1.37,236.0,0.006397,平
7,2454 聯發科,2018-12-17,239,239.0,234,235.0,"2,744","647,453","1,474","1,591,506",13.72,1.38,234.5,-0.002128,平
8,2454 聯發科,2018-12-14,239,240.0,233,236.0,"3,433","811,925","2,512","1,591,506",13.78,1.38,235.0,-0.004237,平
9,2454 聯發科,2018-12-13,233.5,238.0,233.5,237.5,"4,468","1,055,623","1,819","1,591,506",13.86,1.39,236.0,-0.006316,平


In [ ]:
# df_tgg.to_csv('MTK_stock.csv',index=0)

In [11]:
#標註文章漲跌
for day in tqdm(range(len(tnews))):
    targetdate=tnews.loc[day,'post_time'].date()
    for day1 in range(len(df_tgg)):
        try:
            if targetdate==df_tgg.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=df_tgg.loc[day1,'label']
            elif (targetdate+timedelta(1))==df_tgg.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=df_tgg.loc[day1+1,'label']
            elif (targetdate+timedelta(2))==df_tgg.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=df_tgg.loc[day1+1,'label']
            elif (targetdate+timedelta(3))==df_tgg.loc[day1,'年月日'].date():
                tnews.loc[day,'label']=df_tgg.loc[day1+1,'label']
#             elif (targetdate-timedelta(4))==df_tgg.loc[day1,'年月日'].date():
#                 tnews.loc[day,'label']=df_tgg.loc[day1,'label']
#             elif (targetdate-timedelta(5))==df_tgg.loc[day1,'年月日'].date():
#                 tnews.loc[day,'label']=df_tgg.loc[day1,'label']
        except:
            continue

100%|██████████| 5179/5179 [02:02<00:00, 45.61it/s]


In [12]:
tnews.head(3)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,label
0,1451783510043_N01,news,yahoo股市,重大要聞,2016-01-03 08:58:00,《各報要聞》30檔電子股，去年可賺1股本,時報資訊,景氣不佳，電子業成了去年放無薪假的大戶，但在一片不景氣聲中，還是有科技公司逆流而上，初估20...,https://tw.stock.yahoo.com/news_content/url/d/...,跌
1,1451874408573_N01,news,yahoo股市,重大要聞,2016-01-04 09:59:00,台股開紅盤重摔 CES概念撐腰 IC設計鈺創、凌通、迅杰大漲撐盤,鉅亨網,台股今(4)日2016年第一個交易日，開盤不久指數就面臨賣壓襲擊，摜破8200點與月線支撐，...,https://tw.stock.yahoo.com/news_content/url/d/...,跌
2,1451952709026_N01,news,yahoo股市,重大要聞,2016-01-05 07:48:00,《金融》曾銘宗：金融業去年獲利可望再創高,時報資訊,金管會昨（4）日舉辦新春記者會，主委曾銘宗表示，若加計銀行針對中國曝險及不動產的增提準備，2...,https://tw.stock.yahoo.com/news_content/url/d/...,跌


In [13]:
#文章漲跌數

tnews['label'].fillna(value='哈哈', inplace=True)
tnews1=tnews[tnews['label'].str.contains('漲')]
tnews2=tnews[tnews['label'].str.contains('跌')]
tnews3=tnews[tnews['label'].str.contains('平')]
tnews4=tnews[tnews['label'].str.contains('哈哈')]
# tnews5=tnews[tnews['label'].str.contains('不上不下')]
# print(len(tnews),len(tnews1),len(tnews2),len(tnews3),len(tnews4),len(tnews5))
print(len(tnews),len(tnews1),len(tnews2),len(tnews3),len(tnews4))

5179 1593 1399 2143 44


In [14]:
#挑出漲跌平的文章
tnews=tnews[tnews['label'].str.contains('漲|跌|平')]
tnews=tnews.reset_index(drop=True)
len(tnews)

5135

In [15]:
#對每篇文章切字放入cutcontent
corpus = list()
for num in range(len(tnews)):
    content=tnews.loc[num,'content']
    seglist=jieba.cut(content.replace('\n','').replace('，',' '),cut_all=False)
    cutcontent=' '.join(seglist)
    corpus.append(cutcontent)
tnews["cutcontent"]=corpus

Building prefix dict from /home/daniel/code/finance/bda2019_dataset/dict.idkrsi.txt ...
Dumping model to file cache /tmp/jieba.u2e58f25a25b46fee917713c08b4e0329.cache
Loading model cost 0.425 seconds.
Prefix dict has been built succesfully.


In [16]:
tnews.head(3)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,label,cutcontent
0,1451783510043_N01,news,yahoo股市,重大要聞,2016-01-03 08:58:00,《各報要聞》30檔電子股，去年可賺1股本,時報資訊,景氣不佳，電子業成了去年放無薪假的大戶，但在一片不景氣聲中，還是有科技公司逆流而上，初估20...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,景氣 不佳 電子業 成 了 去年 放無薪假 的 大戶 但 在 一片 不景氣 聲中 ...
1,1451874408573_N01,news,yahoo股市,重大要聞,2016-01-04 09:59:00,台股開紅盤重摔 CES概念撐腰 IC設計鈺創、凌通、迅杰大漲撐盤,鉅亨網,台股今(4)日2016年第一個交易日，開盤不久指數就面臨賣壓襲擊，摜破8200點與月線支撐，...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,台股 今 ( 4 ) 日 2016 年 第一個 交易日 開盤 不久 指數 就 面臨 賣壓...
2,1451952709026_N01,news,yahoo股市,重大要聞,2016-01-05 07:48:00,《金融》曾銘宗：金融業去年獲利可望再創高,時報資訊,金管會昨（4）日舉辦新春記者會，主委曾銘宗表示，若加計銀行針對中國曝險及不動產的增提準備，2...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,金管會 昨 （ 4 ） 日 舉辦 新春 記者會 主委 曾 銘宗 表示 若 加計 銀行...


In [17]:
# tnews.to_csv('MTK_content.csv',index=0)

In [18]:
#取前多少個keyword
keywords=list()
with open('bda2019_dataset/companykeyword3000.txt', 'r',encoding='utf-8') as data:
    for keyword in data:
        keywords.append(keyword.strip('\n'))
keywords=keywords[:1500]

# all0431=pd.read_csv("bda2019_dataset/all_0431.csv", encoding="utf-8", header=None)
# all0431=all0431[1].to_numpy()

# 做TfidfVectorizer

In [19]:
#將文章與字典庫做成TFIDF表格
from sklearn.feature_extraction.text import TfidfVectorizer

cv=TfidfVectorizer(vocabulary=keywords) # 預設用只要有空格就一個欄位
# cv=TfidfVectorizer(vocabulary=all0431) # 預設用只要有空格就一個欄位
tfidf=pd.SparseDataFrame(cv.fit_transform(tnews['cutcontent']), 
                       tnews.index,
                       cv.get_feature_names(), 
                       default_fill_value=0)
tfidf=tfidf.drop(columns=['聯發科'])
tfidf.head(7)

,指數,台股,台積電,億元,市場,外資,股價,營收,鴻海,表現,...,股指,超越,顧問處,考慮,之際,吃緊,實質,商品,提高,升值
0,0.000000,0.042315,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048990,...,0.0,0.110506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.032354,0.032933,0.000000,0.000000,0.063210,0.000000,0.230710,0.000000,0.000000,0.152515,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.025018,0.255915,0.027339,0.057616,0.066523,0.000000,0.000000,0.131928,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.066401,0.202772,0.029679,0.033733,0.064865,0.000000,0.039458,0.000000,0.036841,0.039127,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.032650,0.033235,0.000000,0.000000,0.095684,0.033608,0.116412,0.000000,0.000000,0.076956,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.027991,0.024581,0.000000,0.000000,0.000000,0.000000,0.036824,0.030513,0.064813,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.140071,0.040737,0.017887,0.142315,0.019547,0.041195,0.000000,0.026797,0.000000,0.094328,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#把Label放入tfidf表格
tfidf['煞氣的label']=tnews['label']
tfidf.head(7)

,指數,台股,台積電,億元,市場,外資,股價,營收,鴻海,表現,...,超越,顧問處,考慮,之際,吃緊,實質,商品,提高,升值,煞氣的label
0,0.000000,0.042315,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048990,...,0.110506,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
1,0.032354,0.032933,0.000000,0.000000,0.063210,0.000000,0.230710,0.000000,0.000000,0.152515,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
2,0.000000,0.000000,0.025018,0.255915,0.027339,0.057616,0.066523,0.000000,0.000000,0.131928,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
3,0.066401,0.202772,0.029679,0.033733,0.064865,0.000000,0.039458,0.000000,0.036841,0.039127,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
4,0.032650,0.033235,0.000000,0.000000,0.095684,0.033608,0.116412,0.000000,0.000000,0.076956,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
5,0.000000,0.027991,0.024581,0.000000,0.000000,0.000000,0.000000,0.036824,0.030513,0.064813,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌
6,0.140071,0.040737,0.017887,0.142315,0.019547,0.041195,0.000000,0.026797,0.000000,0.094328,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,跌


In [21]:
tfidf2=tfidf.drop(columns=['煞氣的label'])

In [22]:
#把資料級切成train跟test
from sklearn import model_selection
X = tfidf2.to_numpy()
Y = tfidf['煞氣的label'].to_numpy()
validation_size = 0.20
seed = 20
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [24]:
#SVM
from sklearn.svm import SVC

SVC_model = SVC(kernel='linear')
SVC_model.fit(X_train,Y_train)
predictions = SVC_model.predict(X_validation)

print(SVC_model.score(X_train,Y_train))
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.6287731256085687
0.4206426484907498
[[300  70  41]
 [199  89  44]
 [175  66  43]]
              precision    recall  f1-score   support

           平       0.45      0.73      0.55       411
           漲       0.40      0.27      0.32       332
           跌       0.34      0.15      0.21       284

   micro avg       0.42      0.42      0.42      1027
   macro avg       0.39      0.38      0.36      1027
weighted avg       0.40      0.42      0.38      1027



In [ ]:
#SVM K-fold
SVC_model_1 = SVC(kernel='linear')
kfold=model_selection.RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=seed)

result=model_selection.cross_val_score(SVC_model_1, X, Y, cv=kfold, scoring='accuracy')
print(result)
print('%f' % result.mean() )

In [25]:
#Decision tree
from sklearn.tree import DecisionTreeClassifier

decision_model = DecisionTreeClassifier()
decision_model.fit(X_train,Y_train)
predictions1 = decision_model.predict(X_validation)

print(decision_model.score(X_train, Y_train))
print(accuracy_score(Y_validation, predictions1))
print(confusion_matrix(Y_validation, predictions1))
print(classification_report(Y_validation, predictions1))

0.99269717624148
0.3885102239532619
[[202  99 110]
 [129 112  91]
 [112  87  85]]
              precision    recall  f1-score   support

           平       0.46      0.49      0.47       411
           漲       0.38      0.34      0.36       332
           跌       0.30      0.30      0.30       284

   micro avg       0.39      0.39      0.39      1027
   macro avg       0.38      0.38      0.38      1027
weighted avg       0.39      0.39      0.39      1027



In [ ]:
#Decision tree K-fold
decision_model_1 = DecisionTreeClassifier()

result=model_selection.cross_val_score(decision_model_1, X, Y, cv=kfold, scoring='accuracy')
print(result)
print('%f' % result.mean() )

In [26]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification

forest_model=RandomForestClassifier()
forest_model.fit(X_train,Y_train)
predictions2=forest_model.predict(X_validation)

print(forest_model.score(X_train,Y_train))
print(accuracy_score(Y_validation, predictions2))
print(confusion_matrix(Y_validation, predictions2))
print(classification_report(Y_validation, predictions2))

/home/daniel/anaconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9795520934761441
0.41090555014605645
[[264  92  55]
 [185  91  56]
 [140  77  67]]
              precision    recall  f1-score   support

           平       0.45      0.64      0.53       411
           漲       0.35      0.27      0.31       332
           跌       0.38      0.24      0.29       284

   micro avg       0.41      0.41      0.41      1027
   macro avg       0.39      0.38      0.38      1027
weighted avg       0.40      0.41      0.39      1027



In [ ]:
#Random Forest K-fold
forest_model_1 = DecisionTreeClassifier()

result=model_selection.cross_val_score(forest_model_1, X, Y, cv=kfold, scoring='accuracy')
print(result)
print('%f' % result.mean() )

In [27]:
#MLP
from sklearn.neural_network import MLPClassifier

model_MLP = MLPClassifier(hidden_layer_sizes=(200,))
model_MLP.fit(X_train, Y_train)

predictions = model_MLP.predict(X_validation)
print(model_MLP.score(X_train,Y_train))
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.9924537487828627
0.44401168451801365
[[234  92  85]
 [136 128  68]
 [101  89  94]]
              precision    recall  f1-score   support

           平       0.50      0.57      0.53       411
           漲       0.41      0.39      0.40       332
           跌       0.38      0.33      0.35       284

   micro avg       0.44      0.44      0.44      1027
   macro avg       0.43      0.43      0.43      1027
weighted avg       0.44      0.44      0.44      1027



/home/daniel/anaconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
#MLP K-fold
model_MLP_1 = DecisionTreeClassifier()

result=model_selection.cross_val_score(model_MLP_1, X, Y, cv=kfold, scoring='accuracy')
print(result)
print('%f' % result.mean() )

# 做CountVectorizer

In [28]:
#做文字字數的表格
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(vocabulary=keywords) # 預設用只要有空格就一個欄位
# cv=CountVectorizer(vocabulary=all0431) # 預設用只要有空格就一個欄位
tfidf3=pd.SparseDataFrame(cv.fit_transform(tnews['cutcontent']), 
                       tnews.index,
                       cv.get_feature_names(), 
                       default_fill_value=0)
tfidf3=tfidf3.drop(columns=['聯發科'])
tfidf3.head(5)

,指數,台股,台積電,億元,市場,外資,股價,營收,鴻海,表現,...,股指,超越,顧問處,考慮,之際,吃緊,實質,商品,提高,升值
0,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,2,0,6,0,0,4,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,9,1,2,2,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3,2,6,1,1,2,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,3,1,3,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [29]:
#把Label放入tfidf表格
tfidf3['label555']=tnews['label']
tfidf3.head(5)

,指數,台股,台積電,億元,市場,外資,股價,營收,鴻海,表現,...,超越,顧問處,考慮,之際,吃緊,實質,商品,提高,升值,label555
0,0,1,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,跌
1,1,1,0,0,2,0,6,0,0,4,...,0,0,0,0,0,0,0,0,0,跌
2,0,0,1,9,1,2,2,0,0,4,...,0,0,0,0,0,0,0,0,0,跌
3,2,6,1,1,2,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,跌
4,1,1,0,0,3,1,3,0,0,2,...,0,0,0,0,0,0,0,0,0,跌


In [30]:
tfidf4=tfidf3.drop(columns=['label555'])

In [31]:
#把資料級切成train跟test
from sklearn import model_selection
X = tfidf4.to_numpy()
Y = tfidf3['label555'].to_numpy()
validation_size = 0.20
seed = 20
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [32]:
#SVM
SVC_model1 = SVC(kernel='linear')
SVC_model1.fit(X_train,Y_train)
predictions = SVC_model1.predict(X_validation)

print(SVC_model1.score(X_train,Y_train))
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.9016553067185978
0.436222005842259
[[246  76  89]
 [152 114  66]
 [116  80  88]]
              precision    recall  f1-score   support

           平       0.48      0.60      0.53       411
           漲       0.42      0.34      0.38       332
           跌       0.36      0.31      0.33       284

   micro avg       0.44      0.44      0.44      1027
   macro avg       0.42      0.42      0.41      1027
weighted avg       0.43      0.44      0.43      1027



In [33]:
#Decisiontree
decision_model1 = DecisionTreeClassifier()
decision_model1.fit(X_train,Y_train)
predictions1 = decision_model1.predict(X_validation)

print(decision_model1.score(X_train,Y_train))
print(accuracy_score(Y_validation, predictions1))
print(confusion_matrix(Y_validation, predictions1))
print(classification_report(Y_validation, predictions1))

0.99269717624148
0.4186952288218111
[[219 112  80]
 [131 121  80]
 [125  69  90]]
              precision    recall  f1-score   support

           平       0.46      0.53      0.49       411
           漲       0.40      0.36      0.38       332
           跌       0.36      0.32      0.34       284

   micro avg       0.42      0.42      0.42      1027
   macro avg       0.41      0.40      0.40      1027
weighted avg       0.41      0.42      0.41      1027



In [34]:
#random forest
forest_model1=RandomForestClassifier()
forest_model1.fit(X_train,Y_train)
predictions2=forest_model1.predict(X_validation)

print(forest_model1.score(X_train,Y_train))
print(accuracy_score(Y_validation, predictions2))
print(confusion_matrix(Y_validation, predictions2))
print(classification_report(Y_validation, predictions2))

/home/daniel/anaconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9788218111002921
0.4011684518013632
[[270  77  64]
 [183  84  65]
 [153  73  58]]
              precision    recall  f1-score   support

           平       0.45      0.66      0.53       411
           漲       0.36      0.25      0.30       332
           跌       0.31      0.20      0.25       284

   micro avg       0.40      0.40      0.40      1027
   macro avg       0.37      0.37      0.36      1027
weighted avg       0.38      0.40      0.38      1027



In [35]:
#MLP
model_MLP1 = MLPClassifier(hidden_layer_sizes=(100,))
model_MLP1.fit(X_train, Y_train)

predictions = model_MLP1.predict(X_validation)
print(model_MLP1.score(X_train, Y_train))
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.99269717624148
0.433300876338851
[[237  89  85]
 [147 110  75]
 [107  79  98]]
              precision    recall  f1-score   support

           平       0.48      0.58      0.53       411
           漲       0.40      0.33      0.36       332
           跌       0.38      0.35      0.36       284

   micro avg       0.43      0.43      0.43      1027
   macro avg       0.42      0.42      0.42      1027
weighted avg       0.43      0.43      0.43      1027



# 做LSTM

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from keras.optimizers import RMSprop, adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
## 设置字体
from matplotlib.font_manager import FontProperties
fonts = FontProperties(fname = "/Library/Fonts/华文细黑.ttf",size=14)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import re

Using TensorFlow backend.


In [37]:
stopwords=list()
with open('bda2019_dataset/stopword.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))

In [38]:
stnews=tnews[tnews['label'].str.contains('漲|跌|平')]
stnews=stnews.reset_index(drop=True)
stnews.head(3)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,label,cutcontent
0,1451783510043_N01,news,yahoo股市,重大要聞,2016-01-03 08:58:00,《各報要聞》30檔電子股，去年可賺1股本,時報資訊,景氣不佳，電子業成了去年放無薪假的大戶，但在一片不景氣聲中，還是有科技公司逆流而上，初估20...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,景氣 不佳 電子業 成 了 去年 放無薪假 的 大戶 但 在 一片 不景氣 聲中 ...
1,1451874408573_N01,news,yahoo股市,重大要聞,2016-01-04 09:59:00,台股開紅盤重摔 CES概念撐腰 IC設計鈺創、凌通、迅杰大漲撐盤,鉅亨網,台股今(4)日2016年第一個交易日，開盤不久指數就面臨賣壓襲擊，摜破8200點與月線支撐，...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,台股 今 ( 4 ) 日 2016 年 第一個 交易日 開盤 不久 指數 就 面臨 賣壓...
2,1451952709026_N01,news,yahoo股市,重大要聞,2016-01-05 07:48:00,《金融》曾銘宗：金融業去年獲利可望再創高,時報資訊,金管會昨（4）日舉辦新春記者會，主委曾銘宗表示，若加計銀行針對中國曝險及不動產的增提準備，2...,https://tw.stock.yahoo.com/news_content/url/d/...,跌,金管會 昨 （ 4 ） 日 舉辦 新春 記者會 主委 曾 銘宗 表示 若 加計 銀行...


In [39]:
len(stnews)

5135

In [40]:
#把label變成數字 keras的Label不能讀中文
stnews.loc[stnews['label']=='漲', 'label'] = 2
stnews.loc[stnews['label']=='跌', 'label'] = 1
stnews.loc[stnews['label']=='平', 'label'] = 0
stnews.head(10)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,label,cutcontent
0,1451783510043_N01,news,yahoo股市,重大要聞,2016-01-03 08:58:00,《各報要聞》30檔電子股，去年可賺1股本,時報資訊,景氣不佳，電子業成了去年放無薪假的大戶，但在一片不景氣聲中，還是有科技公司逆流而上，初估20...,https://tw.stock.yahoo.com/news_content/url/d/...,1,景氣 不佳 電子業 成 了 去年 放無薪假 的 大戶 但 在 一片 不景氣 聲中 ...
1,1451874408573_N01,news,yahoo股市,重大要聞,2016-01-04 09:59:00,台股開紅盤重摔 CES概念撐腰 IC設計鈺創、凌通、迅杰大漲撐盤,鉅亨網,台股今(4)日2016年第一個交易日，開盤不久指數就面臨賣壓襲擊，摜破8200點與月線支撐，...,https://tw.stock.yahoo.com/news_content/url/d/...,1,台股 今 ( 4 ) 日 2016 年 第一個 交易日 開盤 不久 指數 就 面臨 賣壓...
2,1451952709026_N01,news,yahoo股市,重大要聞,2016-01-05 07:48:00,《金融》曾銘宗：金融業去年獲利可望再創高,時報資訊,金管會昨（4）日舉辦新春記者會，主委曾銘宗表示，若加計銀行針對中國曝險及不動產的增提準備，2...,https://tw.stock.yahoo.com/news_content/url/d/...,1,金管會 昨 （ 4 ） 日 舉辦 新春 記者會 主委 曾 銘宗 表示 若 加計 銀行...
3,1451968008054_N01,news,yahoo股市,重大要聞,2016-01-05 12:05:00,台股盤中－TRF風暴 止穩之路遭金融股扯後腿 8100苦戰,鉅亨網,美股昨天重挫但留下200多點下影線，台股今(5)日開低，最低摔到8072點，一度跌破8100...,https://tw.stock.yahoo.com/news_content/url/d/...,1,美股 昨天 重挫 但 留下 200 多點 下影線 台股 今 ( 5 ) 日 開低 最...
4,1451978807822_N01,news,yahoo股市,重大要聞,2016-01-05 15:07:00,【Y晚報】沙伊交惡！油價只是死貓跳 專家唱衰剩1字頭,Yahoo奇摩股市,美股道瓊昨天連續第三天下挫，收盤大跌276點，不過尾盤拉高收斂跌幅，留下191點的下影線，因...,https://tw.stock.yahoo.com/news_content/url/d/...,1,美股 道瓊 昨天 連續 第三天 下挫 收盤 大跌 276 點 不過 尾盤 拉高 收斂...
5,1452039437423_N01,news,yahoo股市,重大要聞,2016-01-06 08:04:00,《外資》台股電子股，麥格理喊進6檔,時報資訊,麥格理資本證券昨（5）日發表2016年大中華科技產業研究報告，認為產品轉型與新產品週期將重塑...,https://tw.stock.yahoo.com/news_content/url/d/...,1,麥格理 資本 證券 昨 （ 5 ） 日 發表 2016 年 大中華 科技 產業 研究 報告 ...
6,1452070032002_N01,news,yahoo股市,重大要聞,2016-01-06 16:17:00,日盛投顧：台股極短線指數區間7900-8250,Moneydj理財網,1.週三台股受到法人賣超權值股影響，指數開低回測8000，雖在陸股等亞股上漲影響之下，盤中指...,https://tw.stock.yahoo.com/news_content/url/d/...,1,1 . 週三 台股 受到 法人 賣超 權值股 影響 指數 開低 回測 8000 雖 ...
7,1452126731305_N01,news,yahoo股市,重大要聞,2016-01-07 07:58:00,工商時報晨訊,財訊快報,◆新台幣連八貶，創80個月新低，昨收33.453元，農曆封關前可能貶破34元。<BR> ◆人...,https://tw.stock.yahoo.com/news_content/url/d/...,2,◆ 新台幣 連八貶 創 80 個 月 新低 昨收 33.453 元 農曆 封關 ...
8,1452126732568_N01,news,yahoo股市,重大要聞,2016-01-07 07:58:00,經濟日報晨訊,財訊快報,◆陸大降官價，人民幣重貶。<BR> ◆亞幣競貶，台幣跌向33.5元，昨收33.453元，逾六...,https://tw.stock.yahoo.com/news_content/url/d/...,2,◆ 陸大 降官 價 人民幣 重貶 。 < BR > ◆ 亞幣競 貶 台幣 跌向 ...
9,1452135732556_N01,news,yahoo股市,重大要聞,2016-01-07 10:35:00,《台北股市》陸股跳水，台股暴跌百點破10年線,時報資訊,北韓核爆成功、國際原油價格跌跌不休與蘋果手機i6s首季大減產，除引爆歐美主要股市在周三大跌外...,https://tw.stock.yahoo.com/news_content/url/d/...,2,北韓 核爆 成功 、 國際 原油 價格 跌跌 不休 與 蘋果 手機 i6s 首季 大 減產 ...


In [41]:
#把文章重切(扣除掉stopword)放入cutcontent
corpus = list()
for i in range(len(stnews)):
    content=stnews.loc[i,'content']
    content=re.sub('[a-zA-Z0-9\W]', '', content)
    seglist=jieba.cut(content.replace('\n','').replace('，',' '),cut_all=False)
    seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist))
    cutcontent=' '.join(seglist)
    corpus.append(cutcontent)
stnews["cutcontent"]=corpus
stnews.head(3)

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,label,cutcontent
0,1451783510043_N01,news,yahoo股市,重大要聞,2016-01-03 08:58:00,《各報要聞》30檔電子股，去年可賺1股本,時報資訊,景氣不佳，電子業成了去年放無薪假的大戶，但在一片不景氣聲中，還是有科技公司逆流而上，初估20...,https://tw.stock.yahoo.com/news_content/url/d/...,1,景氣 不佳 電子業 成 去年 放無薪假 大戶 一片 不景氣 聲中 科技 公司 逆流 初 估年...
1,1451874408573_N01,news,yahoo股市,重大要聞,2016-01-04 09:59:00,台股開紅盤重摔 CES概念撐腰 IC設計鈺創、凌通、迅杰大漲撐盤,鉅亨網,台股今(4)日2016年第一個交易日，開盤不久指數就面臨賣壓襲擊，摜破8200點與月線支撐，...,https://tw.stock.yahoo.com/news_content/url/d/...,1,台股 今日 第一個 交易日 開盤 不久 指數 面臨 賣壓 襲擊 摜破 點與 月線 支撐 盤面...
2,1451952709026_N01,news,yahoo股市,重大要聞,2016-01-05 07:48:00,《金融》曾銘宗：金融業去年獲利可望再創高,時報資訊,金管會昨（4）日舉辦新春記者會，主委曾銘宗表示，若加計銀行針對中國曝險及不動產的增提準備，2...,https://tw.stock.yahoo.com/news_content/url/d/...,1,金管會 昨日 舉辦 新春 記者會 主委 銘宗 表示 加計 銀行 中國 曝險 不動產 增提 準...


In [42]:
#切成train test 資料
from sklearn import model_selection
X = stnews['cutcontent'].to_numpy()
Y = stnews['label'].to_numpy()
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [43]:
#對label編碼
le = LabelEncoder()
Y_train = le.fit_transform(Y_train).reshape(-1,1)
Y_validation = le.transform(Y_validation).reshape(-1,1)

In [44]:
Y_validation

array([[0],
       [0],
       [0],
       ...,
       [0],
       [2],
       [1]])

In [45]:
#對label做one-hot encoding
ohe = OneHotEncoder()
Y_train = ohe.fit_transform(Y_train).toarray()
Y_validation = ohe.transform(Y_validation).toarray()

/home/daniel/anaconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [46]:
Y_validation

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [47]:
# 使用Tokenizer對詞組進行編碼
## 当我们创建了一个Tokenizer对象后，使用该对象的fit_on_texts()函数，以空格去识别每个词,
## 可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小。
max_words = 50000
max_len = 1000
tok = Tokenizer(num_words=max_words)  ## 使用的最大词语数为5000
tok.fit_on_texts(stnews['cutcontent'])

In [48]:
# 使用word_index属性可以看到每次词对应的编码
# 使用word_counts属性可以看到每个词对应的频数
for ii,iterm in enumerate(tok.word_index.items()):
    if ii < 10:
        print(iterm)
    else:
        break
print("===================")  
for ii,iterm in enumerate(tok.word_counts.items()):
    if ii < 10:
        print(iterm)
    else:
        break

('指數', 1)
('聯發科', 2)
('台股', 3)
('億元', 4)
('台積電', 5)
('元', 6)
('外資', 7)
('市場', 8)
('股價', 9)
('表現', 10)
('景氣', 566)
('不佳', 208)
('電子業', 82)
('成', 469)
('去年', 1742)
('放無薪假', 1)
('大戶', 70)
('一片', 70)
('不景氣', 17)
('聲中', 1)


In [49]:
## 对每个词编码之后，每句新闻中的每个词就可以用对应的编码表示，即每条新闻可以转变成一个向量了：
train_seq = tok.texts_to_sequences(X_train)
val_seq = tok.texts_to_sequences(X_validation)
## 将每个序列调整为相同的长度
train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
val_seq_mat = sequence.pad_sequences(val_seq,maxlen=max_len)

print(train_seq_mat.shape)
print(val_seq_mat.shape)

(4108, 1000)
(1027, 1000)


In [50]:
#建立LSTM模型

model = Sequential()
model.add(Embedding(max_words+1, 256, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu',name='d1'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu',name='d2'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 256)         12800256  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1000, 256)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
d1 (Dense)                   (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
d2 (Dense)           

In [ ]:
model_fit = model.fit(train_seq_mat,Y_train,batch_size=128,epochs=10,
                      validation_data=(val_seq_mat,Y_validation))


Instructions for updating:
Use tf.cast instead.
Train on 4108 samples, validate on 1027 samples
Epoch 1/10
4108/4108 [==============================] - 30s 7ms/step - loss: 1.0866 - acc: 0.4158 - val_loss: 1.0761 - val_acc: 0.4294
Epoch 2/10
4108/4108 [==============================] - 28s 7ms/step - loss: 1.0746 - acc: 0.4187 - val_loss: 1.0783 - val_acc: 0.4119
Epoch 3/10
4108/4108 [==============================] - 28s 7ms/step - loss: 0.9697 - acc: 0.5170 - val_loss: 1.3139 - val_acc: 0.3846
Epoch 4/10
4108/4108 [==============================] - 28s 7ms/step - loss: 0.8446 - acc: 0.5942 - val_loss: 1.2848 - val_acc: 0.4060
Epoch 5/10
4108/4108 [==============================] - 28s 7ms/step - loss: 0.7291 - acc: 0.6580 - val_loss: 1.7331 - val_acc: 0.3690
Epoch 6/10
4108/4108 [==============================] - 28s 7ms/step - loss: 0.6428 - acc: 0.7142 - val_loss: 1.4140 - val_acc: 0.3729
Epoch 7/10
4108/4108 [==============================] - 28s 7ms/step - loss: 0.5570 - acc: 0.7